In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U spacy==2.3.7
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.6
    Uninstalling srsly-2.4.6:
      Successfully uninstalled srsly-2.4.6
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.9
    Uninstalling thinc-8.1.9:
      Successfully uninstalled thinc-8.1.9
  Attempting uninstall: spacy
    Found existing

In [ ]:
import en_core_web_sm

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy
import csv
import random
import time
import numpy as np
import re
import string
from spacy.util import minibatch, compounding
import sys
from spacy import displacy
from itertools import chain

from sklearn.metrics import classification_report

In [ ]:
def load_data_spacy(fname):
  training_data = pd.read_csv(fname, sep=';', encoding='utf-8')
  #train_data.dropna(axis = 0, how ='any',inplace=True)
  #train_data['Num_words_text'] = train_data['text'].apply(lambda x:len(str(x).split())) 
  #mask = train_data['Num_words_text'] >2
  #train_data = train_data[mask]
  print(training_data['author_male'].value_counts())

In [ ]:
corpus="/content/drive/MyDrive/authors_corpus (1) (1).csv"


In [ ]:
training=load_data_spacy(corpus)

True     114
False    113
Name: author_male, dtype: int64


TEXT CLASSIFICATION

In [ ]:
#LOAD DATA
df_authors = pd.read_csv ("/content/drive/MyDrive/authors_corpus (1) (1).csv", sep=";")


In [ ]:
df_authors.head()
#5 examples

,title,author_name,text,author_male
0,Don Quixote,Miguel de Cervantes,"In a village of La Mancha, the name of which I...",True
1,Alice's Adventures in Wonderland,Lewis Carroll,Alice was beginning to get very tired of sitti...,True
2,The Adventures of Huckleberry Finn,Mark Twain,"You don’t know about me, without you have read...",True
3,The Adventures of Tom Sawyer,Mark Twain,The old lady pulled her spectacles down and lo...,True
4,Treasure Island,Robert Louis Stevenson,"Quire Trelawney, Dr. Livesey, and the rest of ...",True


In [ ]:
df_authors.shape
#shape of dataframe

(227, 4)

In [ ]:
#more info
df_authors.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        227 non-null    object
 1   author_name  227 non-null    object
 2   text         227 non-null    object
 3   author_male  227 non-null    bool  
dtypes: bool(1), object(3)
memory usage: 5.7+ KB


TOKENIZING THE DATA

In [ ]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # return preprocessed list of tokens
    return mytokens

REMOVING SPACES & LOWERING WORDS

In [ ]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

VECTORIZING

In [ ]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))


In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)


SPLITTING THE DATA INTO TRAIN AND TEST SETS

In [ ]:
from sklearn.model_selection import train_test_split

X = df_authors['text'] # the features we want to analyze
ylabels = df_authors['author_male'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [ ]:
print(X_train)

45     From the old and pleasantly situated village o...
132    Except for the Marabar Caves—and they are twen...
24     Left Munich at 8:35 P. M., on 1st May, arrivin...
176    A book collector offered Henry Miller a hundre...
203    When everything else has gone from my brain --...
                             ...                        
26     A SQUAT grey building of only thirty-four stor...
25     On the first Monday of the month of April, 162...
71     If you really want to hear about it, the first...
156    I hope I will be able to confide everything to...
19     Among other public buildings in a certain town...
Name: text, Length: 158, dtype: object


In [ ]:
print (y_test)

186    False
17      True
59      True
101     True
216    False
       ...  
197    False
146     True
102    False
53      True
157    False
Name: author_male, Length: 69, dtype: bool


CLASSIFIER

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f44b4068a00>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f4498295b80>)),
                ('classifier', LogisticRegression())])

EVALUATION

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.6521739130434783
Logistic Regression Precision: 0.76
Logistic Regression Recall: 0.5135135135135135
